# <font color='blue'>Análise de Dados com Linguagem Python</font>

## <font color='blue'>Projeto 3</font>
## <font color='blue'>Análise e Limpeza de Dados de Telecomunicações</font>

Uma empresa de telecomunicações possui registros históricos em um volume dedados de  mais  de  150  mil  registros.  

Os  dados  serão  usados  para  modelagem  preditiva,  mas  antes precisamos realizar um trabalho de limpeza pois os dados possuem diversos problemas.

Daremos ênfase ao tratamento devalores ausentes, tratamento de outliers e ajuste dos tipos de dados. 

Ao final vamos gerar um novo arquivo csv com os dados limpos.

## Instalando e Carregando os Pacotes

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
# !pip install -q -U watermark

In [ ]:
# Imports
import math
import sys, os
import numpy as np
import pandas as pd

In [ ]:
# Vamos adicionar caminho para os módulos Python
sys.path.append(os.path.abspath(os.path.join('modulos')))
from estrategia1 import *
from estrategia2 import *
from estrategia3 import *

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Jeferson Oliveira" --iversions

## Carregando os Dados

https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [ ]:
# Criamos uma lista para identificar valores ausentes
lista_labels_valores_ausentes = ["n/a", "na", "undefined"]

In [ ]:
# Carrega o dataset
dataset = pd.read_csv("dados/dataset.csv", na_values = lista_labels_valores_ausentes)

In [ ]:
# Shape
dataset.shape

In [ ]:
# Amostra de dados
dataset.head()

In [ ]:
# Carregando o dicionário de dados
dicionario = pd.read_excel("dados/Dicionario.xlsx")

In [ ]:
# Shape
dicionario.shape

In [ ]:
# Amostra de dados
dicionario.head()

## Análise Exploratória

In [ ]:
# Info
dataset.info()

In [ ]:
# Estatísticas descritivas
dataset.describe()

Não faz sentido calcular estatísticas descritivas para Beared Id, IMSI, MSISDN / Number e IMEI. Mas o método describe() calcula as estatísticas de todas as colunas numéricas. Essas estatísticas estão sendo calculadas antes que os dados sejam limpos. Portanto, pode haver mudanças depois que os valores ausentes e outliers são tratados.

In [ ]:
# Shape
dataset.shape

In [ ]:
# Shape
dicionario.shape

Existem 150.001 linhas e 55 colunas no dataframe. No entanto, temos 56 colunas com seus nomes e descrições no dicionário. Isso significa que há uma coluna descrita, mas não incluída no dataframe. Vamos identificar qual é a coluna faltante.

In [ ]:
# Concatena os dataframes
df_compara_colunas = pd.concat([pd.Series(dataset.columns.tolist()), 
                                dicionario['Fields']], 
                               axis = 1)

In [ ]:
# Renomeia as colunas
df_compara_colunas.rename(columns = {0: 'Coluna no Dataset', 'Fields': 'Coluna no Dicionario'}, 
                          inplace = True)

In [ ]:
# Visualiza
df_compara_colunas

"Dur. (Ms)" é ignorado no dataset como visto no índice 1 em **df_compara_colunas**. É aqui que a ordem das colunas começou a mudar. 

Mas o mesmo nome de coluna "Dur. (Ms)" aparece no dataset no índice 5, enquanto o arquivo de dicionário nos diz que é "Dur. (S)" no índice 6. Como as medidas de ambas as colunas diferem conforme mostrado em seus nomes, nós precisamos verificar qual está certo. Para investigar isso, usaremos a coluna "Dur. (Ms) .1" que se encontra nos índices 28 e 29 no dataset e no arquivo de dicionário, respectivamente.

In [ ]:
dataset[['Dur. (ms)', 'Dur. (ms).1']]

Parece que a coluna "Dur. (Ms)" é medida em segundos. Portanto, vamos renomeá-la apropriadamente. Vamos também renomear algumas das colunas para que fiquem claras como sua descrição e sigam o estilo de nomenclatura de outras colunas.

In [ ]:
# Renomeia colunas
dataset.rename(columns = {'Dur. (ms)': 'Dur (s)', 
                          'Dur. (ms).1': 'Dur (ms)', 
                          'Start ms': 'Start Offset (ms)', 
                          'End ms': 'End Offset (ms)'}, 
               inplace = True)

In [ ]:
# Lista de colunas do dataset
dataset.columns.tolist()

## Estratégia de Limpeza 1 - Tratamento de Valores Ausentes

- 1- Identificando Valores Ausentes
- 2- Drop de Colunas
- 3- Imputação com Preenchimento Reverso
- 4- Imputação com Preenchimento Progressivo
- 5- Imputação de Variáveis Categóricas
- 6- Drop de Linhas

In [ ]:
help(func_calc_percentual_valores_ausentes)

In [ ]:
# Verifica o percentual de valores ausentes
# Função do módulo estratégia 1
func_calc_percentual_valores_ausentes(dataset)

In [ ]:
# Cria tabela com valores ausentes
df_missing = func_calc_percentual_valores_ausentes_coluna(dataset)

In [ ]:
# Visualiza
df_missing

Normalmente removemos colunas com mais de 30% de valores ausentes.

In [ ]:
# Colunas que serão removidas
colunas_para_remover = df_missing[df_missing['% de Valores Ausentes'] >= 30.00].index.tolist()

In [ ]:
# Colunas que serão removidas
colunas_para_remover

Mesmo que as variáveis TCP tenham muitos valores ausentes, em vez de removê-las, iremos imputá-las, uma vez que elas podem ser necessárias para nossa análise posterior.

In [ ]:
# Colunas que serão removidas
colunas_para_remover = [col for col in colunas_para_remover if col not in ['TCP UL Retrans. Vol (Bytes)',
    'TCP DL Retrans. Vol (Bytes)']]

In [ ]:
# Colunas que serão removidas
colunas_para_remover

In [ ]:
# Drop das colunas e cria outro dataframe
dataset_clean = dataset.drop(colunas_para_remover, axis = 1)

In [ ]:
# Shape
dataset_clean.shape

Agora vamos verificar o status dos valores ausentes no dataframe modificado.

In [ ]:
func_calc_percentual_valores_ausentes(dataset_clean)

In [ ]:
func_calc_percentual_valores_ausentes_coluna(dataset_clean)

Uma vez que as porcentagens ausentes de 'TCP UL Retrans. Vol (Bytes) 'e' TCP DL Retrans. Vol (Bytes) 'são muito altos, iremos imputá-los com o método de preenchimento reverso. Nesse caso, usar um único valor como média ou mediana não é aconselhável, pois pode alterar nossos dados de uma forma indesejada, tornando a maioria dos valores igual a um único valor.

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html

In [ ]:
# Imputação com Preenchimento Reverso
fix_missing_bfill(dataset_clean, 'TCP UL Retrans. Vol (Bytes)')

In [ ]:
# Imputação com Preenchimento Reverso
fix_missing_bfill(dataset_clean, 'TCP DL Retrans. Vol (Bytes)')

Avg RTT DL (ms) e Avg RTT UL (ms) têm as próximas porcentagens mais altas de valores ausentes com cerca de 18,5% cada. Vamos verificar se as variáveis estão enviesadas (não seguem uma distribuição normal).

In [ ]:
dataset_clean['Avg RTT DL (ms)'].skew(skipna = True)

In [ ]:
dataset_clean['Avg RTT UL (ms)'].skew(skipna = True)

- Se a assimetria estiver entre -0,5 e 0,5, os dados são bastante simétricos
- Se a assimetria estiver entre -1 e - 0,5 ou entre 0,5 e 1, os dados estão moderadamente inclinados
- Se a assimetria for menor que -1 ou maior que 1, os dados estão altamente enviesados

Visto que ambas as colunas Avg RTT DL (ms) e Avg RTT UL (ms) são fortemente enviesadas positivamente, é aconselhável não imputá-las com sua média. Portanto, usaremos o preenchimento progressivo.

In [ ]:
# Imputação com Preenchimento Progressivo
fix_missing_ffill(dataset_clean, 'Avg RTT DL (ms)')

In [ ]:
# Imputação com Preenchimento Progressivo
fix_missing_ffill(dataset_clean, 'Avg RTT UL (ms)')

> Checamos novamente os valores ausentes.

In [ ]:
func_calc_percentual_valores_ausentes(dataset_clean)

In [ ]:
func_calc_percentual_valores_ausentes_linha(dataset_clean)

In [ ]:
func_calc_percentual_valores_ausentes_coluna(dataset_clean)

In [ ]:
dataset_clean.info()

Visto que "Handset Type" e "Handset Manufacturer" são colunas categóricas, é melhor imputá-los com o valor "unknown" para que não enviesemos os dados.

In [ ]:
# Imputação de variáveis categóricas
fix_missing_value(dataset_clean, 'Handset Type', 'unknown')
fix_missing_value(dataset_clean, 'Handset Manufacturer', 'unknown')

> Checamos novamente os valores ausentes.

In [ ]:
func_calc_percentual_valores_ausentes(dataset_clean)

In [ ]:
func_calc_percentual_valores_ausentes_linha(dataset_clean)

Uma vez que apenas 2,08% das linhas contêm pelo menos um valor ausente e o número total de linhas é de cerca de 150000, descartar essas linhas não terá um impacto negativo perceptível.

In [ ]:
# Drop de linhas com valores ausentes
drop_rows_with_missing_values(dataset_clean)

In [ ]:
func_calc_percentual_valores_ausentes(dataset_clean)

In [ ]:
# Shape
dataset_clean.shape

## Estratégia 2 - Conversão de Tipos de Dados

In [ ]:
dataset_clean.dtypes

In [ ]:
dataset_clean

Observando as colunas, podemos perceber que as colunas "Start" e "End" são, na verdade, valores de data e hora, embora sejam rotuladas como objetos pelo pandas. Além dessas duas colunas, todas as outras colunas com tipos de dados de objeto são, na verdade, valores de string. Portanto, vamos converter essas colunas em seus tipos de dados apropriados.

In [ ]:
# Converte para datetime
convert_to_datetime(dataset_clean, ['Start', 'End'])

In [ ]:
# Extrai as colunas do tipo object
string_columns = dataset_clean.select_dtypes(include = 'object').columns.tolist()

In [ ]:
# Visualiza
string_columns

In [ ]:
# Converte para string
convert_to_string(dataset_clean, string_columns)

Também sabemos que Bearer Id, IMSI, MSISDN / Number, IMEI são números únicos usados para identificação. Portanto, para melhor legibilidade (e facilitar os filtros usados mais frente), vamos alterá-los de float64 para int64.

In [ ]:
# Lista de colunas para conversão
int_cols = ['Bearer Id', 'IMSI', 'MSISDN/Number', 'IMEI',]

In [ ]:
# Converte para int
convert_to_int(dataset_clean, int_cols)

In [ ]:
dataset_clean.dtypes

In [ ]:
# Vamos checar se há registros duplicados
drop_duplicates(dataset_clean)

Como vimos na seção de limpeza da coluna, temos duas colunas de duração, uma em segundos e a outra em microssegundos. Vamos verificar se os valores são iguais convertendo os microssegundos em segundos.

In [ ]:
# Conversão e comparação
temp_df = dataset_clean[['Dur (s)', 'Dur (ms)']].copy()
multiply_by_factor(temp_df, ['Dur (ms)'], 1/1000)
temp_df['comparison'] = (temp_df['Dur (s)'] == temp_df['Dur (ms)'].apply(math.floor))

In [ ]:
temp_df

In [ ]:
# As duas colunas são iguais?
print(all(temp_df['comparison']))

Isso prova que, quando arredondadas, essas duas colunas são iguais. Portanto, manteremos "Dur (ms)", pois é mais preciso, e removeremos "Dur (s)".

In [ ]:
# Drop de coluna
drop_columns(dataset_clean, ['Dur (s)'])

## Estratégia 3 - Tratamento de Outliers

![title](imagens/outliers.png)

In [ ]:
# Cria o objeto trata outlier
trata_outlier = TrataOutlier(dataset_clean)

In [ ]:
# Lista de colunas float64
lista_colunas = dataset_clean.select_dtypes('float64').columns.tolist()

In [ ]:
lista_colunas

In [ ]:
# Visão geral dos outliers
trata_outlier.getOverview(lista_colunas)

In [ ]:
# Replace dos outliers
trata_outlier.replace_outliers_with_fences(lista_colunas)

In [ ]:
# Visão geral dos outliers
trata_outlier.getOverview(lista_colunas)

## Bônus: Gerando Novas Colunas

Ter a soma dos volumes de dados de upload e download para cada aplicativo como um total pode ser necessário para análises?

In [ ]:
dataset_clean['Social Media Data Volume (Bytes)'] = dataset_clean['Social Media UL (Bytes)'] + dataset_clean['Social Media DL (Bytes)']

In [ ]:
dataset_clean['Google Data Volume (Bytes)'] = dataset_clean['Google UL (Bytes)'] + dataset_clean['Google DL (Bytes)']

In [ ]:
dataset_clean['Email Data Volume (Bytes)'] = dataset_clean['Email UL (Bytes)'] + dataset_clean['Email DL (Bytes)']

In [ ]:
dataset_clean['Youtube Data Volume (Bytes)'] = dataset_clean['Youtube UL (Bytes)'] + dataset_clean['Youtube DL (Bytes)']

In [ ]:
dataset_clean['Netflix Data Volume (Bytes)'] = dataset_clean['Netflix UL (Bytes)'] + dataset_clean['Netflix DL (Bytes)']

In [ ]:
dataset_clean['Gaming Data Volume (Bytes)'] = dataset_clean['Gaming UL (Bytes)'] + dataset_clean['Gaming DL (Bytes)']

In [ ]:
dataset_clean['Other Data Volume (Bytes)'] = dataset_clean['Other UL (Bytes)'] + dataset_clean['Other DL (Bytes)']

In [ ]:
dataset_clean['Total Data Volume (Bytes)'] = dataset_clean['Total UL (Bytes)'] + dataset_clean['Total DL (Bytes)']

In [ ]:
dataset_clean.info()

In [ ]:
dataset_clean.shape

## Salvando os Dados Após a Limpeza

In [ ]:
# Salvando os dados
dataset_clean.to_csv('dados/dataset_clean.csv')

# Fim